PDF READER APPLICATION MADE 

PDF CONTENT IS MADE INTO CHUNKS (TEXT CHUNKS)
USE AN OOPEN AI EMBEDDINGS 
THIS IS USED TO CINVERT IT INTO VECTORS 
THIS VECTOR IS USED IN VECTOR SEARCH DB 
WITH VECTOR DB WE CAN DO SIMILARITY SEARCH 
VECTOR SEARCH DB USED HERE IS PINECONE


In [12]:
import openai
import langchain
import pinecone 
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI

In [13]:
from dotenv import load_dotenv
import os

In [14]:
load_dotenv()

True

In [15]:
def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents

In [17]:
doc=read_doc('documents/')
len(doc)

196

In [ ]:
## Divide the docs into chunks
def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return docs

In [19]:
documents=chunk_data(docs=doc)
len(documents)


196

In [ ]:
embeddings=OpenAIEmbeddings(api_key=os.environ['YOUR_OPENAI_API_KEY'] , model="text-embedding-ada-002")
embeddings

In [ ]:
vectors=embeddings.embed_query("what is life")
len(vectors)

In [31]:
from pinecone import Pinecone as PineconeClient, ServerlessSpec  # New Pinecone class
pc = PineconeClient(api_key="923d5299-ab4c-4407-bfe6-7f439d9a9cb9", environment="gcp-starter")

In [34]:
## Vector Search DB In Pinecone
# pinecone.init(
#     api_key="923d5299-ab4c-4407-bfe6-7f439d9a9cb9",
#     environment="gcp-starter"
# )
index_name="langchainvector"
# docsearch = Pinecone.from_documents(documents=doc, embedding=embeddings, index_name=index_name)
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        spec=ServerlessSpec(
            cloud="gcp",
            region="us-central1"  # You must specify the correct region for your Pinecone project
            # dimension=1536  # Must match the dimension of your embedding model
        )
    )

# Get the index
index = pc.Index(index_name)

# Initialize embedding model
embeddings = OpenAIEmbeddings()

# ✅ Important: Pass the actual Pinecone index object
from langchain.vectorstores import Pinecone as LangchainPinecone

docsearch = LangchainPinecone.from_documents(
    documents=doc,
    embedding=embeddings,
    index=index  # Pass the Pinecone SDK v3 index object here
)


PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-01', 'x-cloud-trace-context': 'b87d4eddd86d64e007c0ed9512fe12e0', 'date': 'Mon, 19 May 2025 04:57:40 GMT', 'server': 'Google Frontend', 'Content-Length': '107', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"INVALID_ARGUMENT","message":"Dimension is required for dense vector index"},"status":400}


In [ ]:
## Cosine Similarity Retreive Results from VectorDB
def retrieve_query(query,k=2):
    matching_results=index.similarity_search(query,k=k)
    return matching_results

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI

In [ ]:
llm=OpenAI(model_name="text-davinci-003",temperature=0.5)
chain=load_qa_chain(llm,chain_type="stuff")

In [ ]:

## Search answers from VectorDB
def retrieve_answers(query):
    doc_search=retrieve_query(query)
    print(doc_search)
    response=chain.run(input_documents=doc_search,question=query)
    return response


our_query = "How much the agriculture target will be increased by how many crore?"
answer = retrieve_answers(our_query)
print(answer)